In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/My Drive/kaggle/bengali'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/content/drive/My Drive/kaggle/bengali/nbs/submit-test-new-formula.ipynb


In [3]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [4]:
import tensorflow as tf
import keras
from keras.layers import LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm.auto import tqdm
import cv2

Using TensorFlow backend.


In [0]:
def build_model(model_seq, need_compile=False):  
    x_in = tf.keras.layers.Input(shape=(64, 64, 1))
    x = model_seq(x_in)
    out_grapheme = tf.keras.layers.Dense(168, activation='softmax', name='grapheme')(x)
    out_vowel = tf.keras.layers.Dense(11, activation='softmax', name='vowel')(x)
    out_consonant = tf.keras.layers.Dense(7, activation='softmax', name='consonant')(x)
    
    model = tf.keras.Model(inputs=x_in, outputs=[out_grapheme, out_vowel, out_consonant])
    
    if need_compile==True:model.compile(optimizer=RMSprop(lr=initial_learningrate), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
model_seq = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', input_shape=(64, 64, 1)),
    tf.keras.layers.BatchNormalization(momentum=0.5, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Conv2D(filters=64,  kernel_size=(3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),

    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),    
    
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same'),
    tf.keras.layers.BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform"),
    tf.keras.layers.LeakyReLU(alpha=0.1),

    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    tf.keras.layers.BatchNormalization(),    
])

In [0]:
model = build_model(model_seq)

In [0]:
model.load_weights('/kaggle/input/eps80-wloss/eps80_wloss.h5')

In [0]:
SIZE=128
HEIGHT = 137
WIDTH = 236

def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax

def crop_resize(img0, size=SIZE, pad=16):
    #crop a box around pixels large than the threshold 
    #some images contain line at the sides
    ymin,ymax,xmin,xmax = bbox(img0[5:-5,5:-5] > 80)
    #cropping may cut too much, so we need to add it back
    xmin = xmin - 13 if (xmin > 13) else 0
    ymin = ymin - 10 if (ymin > 10) else 0
    xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
    ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
    img = img0[ymin:ymax,xmin:xmax]
    #remove lo intensity pixels as noise
    img[img < 28] = 0
    lx, ly = xmax-xmin,ymax-ymin
    l = max(lx,ly) + pad
    #make sure that the aspect ratio is kept in rescaling
    img = np.pad(img, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')
    return cv2.resize(img,(size,size))

In [0]:
def test_batch_generator(frame, batch_size=64):    
    
    num_imgs = len(frame)
    stats = (0.0692, 0.2051)
    
    for batch_start in range(0, num_imgs,batch_size):   
        
            cur_batch_size = min(num_imgs,batch_start+batch_size)-batch_start
            
            idx = np.arange(batch_start,batch_start+cur_batch_size)
            names_batch = frame.iloc[idx,0].values
            imgs_batch = 255 - frame.iloc[idx, 1:].values.reshape(-1, HEIGHT, WIDTH,1).astype(np.uint8)
            
            resized_imgs_batch = np.zeros((cur_batch_size,64,64,1))
            
            for j in range(cur_batch_size):
                img = (imgs_batch[j,:,:,0]*(255.0/imgs_batch[j,:,:,0].max())).astype(np.uint8)
                img = crop_resize(img)
                img = (img.astype(np.float32)/255.0 - stats[0])/stats[1]
                img = cv2.resize(img,(64,64)).reshape(64,64,1)
                resized_imgs_batch[j,] = img
                
            yield resized_imgs_batch,names_batch

In [0]:
P_TEST_MODE=True

TEST_F = ['/kaggle/input/bengaliaicv19feather/test_image_data_0.feather',
          '/kaggle/input/bengaliaicv19feather/test_image_data_1.feather',
          '/kaggle/input/bengaliaicv19feather/test_image_data_2.feather',
          '/kaggle/input/bengaliaicv19feather/test_image_data_3.feather']

TEST_P = ['/kaggle/input/bengaliai-cv19/test_image_data_0.parquet',
          '/kaggle/input/bengaliai-cv19/test_image_data_1.parquet',
          '/kaggle/input/bengaliai-cv19/test_image_data_2.parquet',
          '/kaggle/input/bengaliai-cv19/test_image_data_3.parquet']

if P_TEST_MODE==True: TEST = TEST_P
else                : TEST = TEST_F

batch_size=1024
row_id,target = [],[]

for fname in TEST:
    if P_TEST_MODE == True: frame = pd.read_parquet(fname) 
    else:                   frame = pd.read_feather(fname)
        
    test_gen = test_batch_generator(frame,batch_size=batch_size)
    
    for batch_x,batch_name in tqdm(test_gen):
            batch_predict = model.predict(batch_x)
            for idx,name in enumerate(batch_name):
                row_id += [f'{name}_consonant_diacritic',f'{name}_grapheme_root',f'{name}_vowel_diacritic']
                target += [ np.argmax(batch_predict[2],axis=1)[idx],
                            np.argmax(batch_predict[0],axis=1)[idx],
                            np.argmax(batch_predict[1],axis=1)[idx]]
                
    frame.drop(frame.index.values,inplace=True)
    
sub_df = pd.DataFrame({'row_id': row_id, 'target': target})
sub_df.to_csv('submission.csv', index=False)
sub_df.head()

In [0]:
sub_df